# Get dataset

In [19]:
import pandas as pd
import numpy as np
import random
import re  
%load_ext autoreload
%autoreload 2

## Templates

In [2]:
# templates
templates = pd.read_json("data/templates.json")
init_uttr_updated = pd.read_csv("data/InitUtterance_list_updated.csv")
templates_updated = templates.merge(init_uttr_updated, on='id')
templates_updated = templates_updated[['id','de', 'fr', 'it', 'en']]
templates_updated.head()

,id,de,fr,it,en
0,MyCloud.OpenArea.Init.Utterance,{'texts': [{'ttsText': '{MyCloudArea} auf #myC...,{'texts': [{'ttsText': 'afficher {MyCloudArea}...,{'texts': [{'ttsText': 'apri {MyCloudArea} su ...,{'texts': [{'ttsText': 'show {MyCloudArea} on ...
1,Router.SwitchOnWiFi.Init.Utterance,"{'texts': [{'ttsText': 'aktiviere das Wlan'}, ...","{'texts': [{'ttsText': 'active le Wi-Fi'}, {'t...","{'texts': [{'ttsText': 'attiva il #wifi'}, {'t...","{'texts': [{'ttsText': 'activate the Wi-Fi'}, ..."
2,Tv.GenericExitToTv.Init.Utterance,{'texts': [{'ttsText': 'gehe zurück zum Fernse...,"{'texts': [{'ttsText': 'revenir à la télé'}, {...","{'texts': [{'ttsText': 'torna alla TV'}, {'tts...","{'texts': [{'ttsText': 'back to TV'}, {'ttsTex..."
3,Router.SwitchOffWiFi.Init.Utterance,{'texts': [{'ttsText': 'deaktiviere das Wlan'}...,"{'texts': [{'ttsText': 'désactiver le Wi-Fi'},...","{'texts': [{'ttsText': 'disattiva il wifi'}, {...",{'texts': [{'ttsText': 'can you please switch ...
4,Tv.RadioChannelChange.Init.Utterance,{'texts': [{'ttsText': 'auf Radio {RadioChanne...,{'texts': [{'ttsText': 'prendre radio {RadioCh...,{'texts': [{'ttsText': 'metti radio {RadioChan...,{'texts': [{'ttsText': 'can you turn on radio ...


In [3]:
templates_updated.describe()

,id,de,fr,it,en
count,39,39,39,39,39
unique,39,39,39,39,39
top,Tv.VolumeDown.Init.Utterance,{'texts': [{'ttsText': '{SmartHomeSceneName} S...,{'texts': [{'ttsText': 'activer scène {SmartHo...,{'texts': [{'ttsText': 'attiva scena {SmartHom...,{'texts': [{'ttsText': 'activate {SmartHomeSce...
freq,1,1,1,1,1


## Entities

In [4]:
# entities
entities = pd.read_json("data/entities.json")
entities = entities[['value', 'type', 'language', 'normalizedValue', 'aliases']]
entities.head()

,value,type,language,normalizedValue,aliases
0,Widnau,LocalsearchLocation,de,widnau,[]
1,Zurigo,LocalsearchLocation,it,zurigo,[]
2,Solothurn,LocalsearchLocation,de,solothurn,[]
3,Muttenz,LocalsearchLocation,de,muttenz,[]
4,Lyss,LocalsearchLocation,de,lyss,[]


In [5]:
entities.describe()

,value,type,language,normalizedValue,aliases
count,65471,65471,65471,65471,65471
unique,51272,33,4,52082,3793
top,Golf,SeriesName,de,golf,[]
freq,20,17913,22746,20,59649


# Explore data

### Entities catagorical count

In [6]:
df = entities[['type', 'language']]
df = df.groupby(['type','language']).size().reset_index(name='count') 
df.pivot(index='type', columns='language', values='count')#.plot(kind='bar')

language,de,en,fr,it
type,,,,
AlbumName,NaN,1479.0,NaN,NaN
AppName,326.0,273.0,312.0,290.0
AreaName,26.0,26.0,26.0,26.0
ArtistName,NaN,822.0,NaN,NaN
BroadcastName,715.0,483.0,676.0,663.0
CallingNumberType,3.0,4.0,4.0,4.0
ClipName,353.0,320.0,342.0,338.0
DossierName,8.0,3.0,5.0,3.0
FirstName,952.0,NaN,NaN,NaN


### Generate correct command

In [58]:
from generator.tools import filter_aliases
l = [   "SRF 3",
		"D R S 3",
		"Schweiz 3",
		"S R F 3",
		"DRS 3",
        "s r f 3",
		"s r f drei",
		"d r s drei",
		"schweiz drei",
		"DRS drei",
        "one and a half man"
	]
l2 = [
		"D M A X",
		"d max",
		"DMAX",
		"d m a x",
		"D Max",
		"D Max Italia",
		"d max italia"
	]
l3 = [
		"TF three",
		"TF 3",
		"T F 3",
		"t f three"
	]
l4 = [
		"E V Z Academy",
		"e v z academy"
	]
l5 = [
		"BN",
		"b n t v",
		"B N",
		"b n",
		"BN TV",
		"B N T V"
	]
filter_aliases(l5)

[&#39;BN&#39;, &#39;BN TV&#39;, &#39;BNTV&#39;]

In [59]:
l6 = [
		"S R F",
		"Schweiz 1",
		"SRF eis",
		"DRS 1",
		"SRF",
		"S R F eis",
		"s r f einz",
		"d r s eins",
		"S R",
		"schweizer",
		"Schweizer",
		"DRS eins",
		"SR",
		"SRF einz",
		"D R S",
		"s r f eis",
		"SRF 1",
		"D R S 1",
		"DRS",
		"s r",
		"S R F 1",
		"s r f eins",
		"SRF eins",
		"d r s",
		"S R F einz",
		"s r f",
		"schweiz eins"
	]
filter_aliases(l6)

[&#39;Schweiz 1&#39;,
 &#39;SRF eis&#39;,
 &#39;DRS 1&#39;,
 &#39;SRF&#39;,
 &#39;schweizer&#39;,
 &#39;Schweizer&#39;,
 &#39;DRS eins&#39;,
 &#39;SR&#39;,
 &#39;SRF einz&#39;,
 &#39;SRF 1&#39;,
 &#39;DRS&#39;,
 &#39;SRF eins&#39;,
 &#39;schweiz eins&#39;]

In [61]:
Normalizer().normalize_text('1', '')

&#39;un&#39;

In [62]:
from generator.module import Generator
from generator.normalizer import Normalizer
gen = Generator(templates=templates_updated, entities=entities)
gen.get_command(target_id='Tv.RadioChannelChange.Init.Utterance', 
                target_lang='de', 
                verbose=True)

Choose template: 
	auf Radio {RadioChannelName} schalten
After tag removal: 
	auf Radio my 105 Fresh schalten
After normalizer: 
	auf radio my einhundertfünf fresh schalten


&#39;auf radio my einhundertfünf fresh schalten&#39;

In [18]:

gen.get_command(target_id='MyCloud.OpenArea.Init.Utterance', 
                    target_lang='fr', 
                    normalizer= Normalizer().normalize_text,
                    verbose=True)

Choose template: 
	j&#39;ai envie de voir {MyCloudArea} sur #myCloud
After tag removal: 
	j&#39;ai envie de voir albums sur #myCloud
After normalizer: 
	j&#39;ai envie de voir albums sur mycloud


&quot;j&#39;ai envie de voir albums sur mycloud&quot;

In [17]:
normalizer = 
gen = Generator(templates=templates_updated, entities=entities)

gen.generate_command(target_id='Tv.GenericExitToTv.Init.Utterance', 
                    target_lang='de', 
                    normalizer= normalizer)

'wieder fernsehen'

In [ ]:
pd.DataFrame()